In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras

In [ ]:
import os

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [ ]:
import io

In [ ]:
lines = io.open(path_to_file, encoding='UTF-8').read().strip().split('\n')



In [ ]:
spanish=[]
english=[]

In [ ]:
for l in lines[:30000]:
  d=l.split('\t')
  english.append(d[0])
  spanish.append(d[1])

In [ ]:
import re

In [ ]:
def preprocess(w):
  w = w.lower()
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w 
  return w


In [ ]:
def preprocess1(w):
  w = w.lower()
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  return w

In [ ]:
eng=list(map(preprocess1,english))

In [ ]:
tar=list(map(preprocess1,spanish))
spa=list(map(preprocess,spanish))

In [ ]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts(eng)

In [ ]:
a=tokenizer.texts_to_sequences(eng)

In [ ]:
train_data=tf.keras.preprocessing.sequence.pad_sequences(a,padding='post')

In [ ]:
train_data.shape

(30000, 25)

In [ ]:
tokenizer.fit_on_texts(spa)
b=tokenizer.texts_to_sequences(spa)
test_data=tf.keras.preprocessing.sequence.pad_sequences(b,padding='post')

In [ ]:
tokenizer.fit_on_texts(tar)
c=tokenizer.texts_to_sequences(tar)
tar_data=tf.keras.preprocessing.sequence.pad_sequences(c,padding='post')

In [ ]:
def one(i):
  return tf.one_hot(i,depth=34)

In [ ]:
def one1(i):
  return tf.one_hot(i,depth=33)

In [ ]:
test=list(map(one ,test_data))

In [ ]:
train=list(map(one1 ,train_data))
target=list(map(one,tar_data))

In [ ]:
encoder_inputs = keras.Input(shape=(None, 33))
encoder = keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, 34))
decoder_lstm = keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(34, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile( optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)


In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 34)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 296960      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  297984      input_2[0][0]                    
                                                                 lstm[0][1]            

In [ ]:
model.fit(
    [train, test],
    target,
    batch_size=16,
    epochs=10,
    validation_split=0.2,
)